In [ ]:
# general and data handling
import numpy as np
import pandas as pd
import os
from collections import Counter

In [ ]:
#current directory
os

In [ ]:
# Required RDKit modules
import rdkit as rd
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import RDConfig
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem import AllChem 
from rdkit.Chem.rdMolDescriptors import GetAtomPairFingerprint
from rdkit.Chem.AtomPairs import Torsions
from rdkit.Chem import Draw
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdmolops
from rdkit.Chem import rdMolDescriptors

In [ ]:
# modeling
import sklearn as sk
from sklearn import metrics

In [ ]:
# Graphing
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#loading sdfFile
sdfFile = os.path.join(RDConfig.RDDataDir, 'C:\\Users\\ADMIN\\Desktop\\structures.molV3.sdf')

In [ ]:
#adding columns to pandas
dframe = PandasTools.LoadSDF(sdfFile,smilesName='SMILES',molColName='Molecule', includeFingerprints=True)

In [ ]:
#table content
dframe

In [ ]:
#table columns
dframe.columns

In [ ]:
# drop columns from pandas 
data=dframe.drop(['ID', 'PREFERRED_NAME', 'CAS_RN', 'SYNONYMS', 'URL','Molecule', 'type', 'CAS', 'Name', 'Ambiguous'],axis=1)

In [ ]:
#reading mol from smiles
data["mol"] = [Chem.MolFromSmiles(x) for x in dframe["SMILES"]]

In [ ]:
#count the nummber of unique rows in the SMILES column
data['SMILES'].nunique()

In [ ]:
#Compare that with total rows
data['SMILES'].count()

In [ ]:
#we can drop the duplicated ones
data=data.drop_duplicates(['SMILES'])
display(data)

In [ ]:
#atompair fingerprint(Generate BitVect fingerprints)
MinLength=1
MaxLength=7
FPSize = 2048
BitsPerHash = 2       
data["atom_fp"] = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(x, minLength = MinLength, maxLength = MaxLength, includeChirality = True, nBits = FPSize, nBitsPerEntry = BitsPerHash) for x in data['mol']]

In [ ]:
data["atom_fp"] 

In [ ]:
atom_fp = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(x, minLength = MinLength, maxLength = MaxLength, includeChirality = True) for x in data['mol']]

In [ ]:
atom_fp

In [ ]:
len(atom_fp[1])

In [ ]:
print(atom_fp[1].GetNumBits())
print(atom_fp[1].GetNumOffBits())
print(atom_fp[1].GetNumOnBits())
print(atom_fp[1].ToBinary())

In [ ]:
# convert the RDKit explicit vectors into numpy arrays
atom_fp_np = []
for fp in atom_fp:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  atom_fp_np.append(arr)

In [ ]:
#tanimota similarity for atompair fingerprint
data['atom_tanimoto'] = DataStructs.BulkTanimotoSimilarity(atom_fp[0], atom_fp)
data['atom_tanimoto']

In [ ]:
#drawing molstructure for tanimoto values
Draw.MolsToGridImage(data["mol"], molsPerRow=5, subImgSize=(200,200), legends=['Tanimoto: {:.2f}'.format(i) for i in data['atom_tanimoto']])

In [ ]:
#MACCSKEY
data["mac_fp"]=[rdMolDescriptors.GetMACCSKeysFingerprint(x) for x in data['mol']]

In [ ]:
data["mac_fp"]

In [ ]:
mac_fp =[rdMolDescriptors.GetMACCSKeysFingerprint(x) for x in data['mol']]

In [ ]:
mac_fp

In [ ]:
len(mac_fp[1])

In [ ]:
print(mac_fp[1].GetNumBits())
print(mac_fp[1].GetNumOffBits())
print(mac_fp[1].GetNumOnBits())
print(mac_fp[1].ToBinary())

In [ ]:
# convert the RDKit explicit vectors into numpy arrays
mac_fp_np = []
for fp in mac_fp:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  mac_fp_np.append(arr)

In [ ]:
#tanimota similarity for maccskey fingerprint
data['mac_tanimoto'] = DataStructs.BulkTanimotoSimilarity(mac_fp[0], mac_fp)
data['mac_tanimoto'] 

In [ ]:
#from maccskey tanimoto values
Draw.MolsToGridImage(data["mol"], molsPerRow=5, subImgSize=(200,200), legends=['Tanimoto: {:.2f}'.format(i) for i in data['mac_tanimoto']])

In [ ]:
#Morgan fingerprint
#create a "morg_fp" column
data["morg_fp"] = [AllChem.GetMorganFingerprintAsBitVect(x, 2,  nBits = 2048) for x in data['mol']]

In [ ]:
data["morg_fp"]

In [ ]:
# generate morgan fingeprints with radius 2 contained in a list
morg_fp = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits = 2048) for x in data['mol']]

In [ ]:
morg_fp

In [ ]:
len(morg_fp[1])

In [ ]:
print(morg_fp[1].GetNumBits())
print(morg_fp[1].GetNumOffBits())
print(morg_fp[1].GetNumOnBits())
print(morg_fp[1].ToBinary())

In [ ]:
# convert the RDKit explicit vectors into numpy arrays
morg_fp_np = []
for fp in morg_fp:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  morg_fp_np.append(arr)

In [ ]:
morg_fp_np

In [ ]:
#tanimota similarity for morgan fingerprint
data['morg_tanimoto'] = DataStructs.BulkTanimotoSimilarity(morg_fp[0], morg_fp)
data['morg_tanimoto'] 

In [ ]:
# drawing images from morgan tanimoto values
Draw.MolsToGridImage(data["mol"], molsPerRow=5, subImgSize=(200,200), legends=['Tanimoto: {:.2f}'.format(i) for i in data['morg_tanimoto']])

In [ ]:
#pathlength or daylight fingerprint
data['palen_fp'] = [FingerprintMols.FingerprintMol(x, minPath=1, maxPath=7, fpSize=2048, bitsPerHash=2, useHs=True, tgtDensity=0.0,
                               minSize=64 ) for x in data['mol']]

In [ ]:
data['palen_fp']

In [ ]:
palen_fp =  [FingerprintMols.FingerprintMol(x, minPath=1, maxPath=7, fpSize=2048, bitsPerHash=2, useHs=True, tgtDensity=0.0, minSize=64,) for x in data['mol']]

In [ ]:
palen_fp

In [ ]:
len(palen_fp[1])

In [ ]:
print(palen_fp[1].GetNumBits())
print(palen_fp[1].GetNumOffBits())
print(palen_fp[1].GetNumOnBits())
print(palen_fp[1].ToBinary())

In [ ]:
# convert the RDKit explicit vectors into numpy arrays
palen_fp_np = []
for fp in palen_fp:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  palen_fp_np.append(arr)

In [ ]:
palen_fp_np

In [ ]:
#tanimota similarity for path_based fingerprint
data['palen_tanimoto'] = DataStructs.BulkTanimotoSimilarity(palen_fp[0], palen_fp)
data['palen_tanimoto']

In [ ]:
#drawing molstructure for tanimoto values
Draw.MolsToGridImage(data["mol"], molsPerRow=5, subImgSize=(200,200), legends=['Tanimoto: {:.2f}'.format(i) for i in data['palen_tanimoto']])

In [ ]:
#Topological fingerprint
FPSize = 2048
BitsPerHash = 4 
data["toptor_fp"] = [rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(x,  includeChirality = True, nBits = FPSize, nBitsPerEntry = BitsPerHash) for x in data['mol']]

In [ ]:
data["toptor_fp"]

In [ ]:
toptor_fp = [rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(x,  includeChirality = True, nBits = FPSize, nBitsPerEntry = BitsPerHash) for x in data['mol']]

In [ ]:
toptor_fp

In [ ]:
len(toptor_fp[1])

In [ ]:
print(toptor_fp[1].GetNumBits())
print(toptor_fp[1].GetNumOffBits())
print(toptor_fp[1].GetNumOnBits())
print(toptor_fp[1].ToBinary())

In [ ]:
# convert the RDKit explicit vectors into numpy arrays
toptor_fp_np = []
for fp in toptor_fp:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  toptor_fp_np.append(arr)

In [ ]:
toptor_fp_np

In [ ]:
#tanimota similarity for topological fingerprint
data['toptor_tanimoto'] = DataStructs.BulkTanimotoSimilarity(toptor_fp[0], toptor_fp)
data['toptor_tanimoto']

In [ ]:
#drawing molstructure for tanimoto values
Draw.MolsToGridImage(data["mol"], molsPerRow=5, subImgSize=(200,200), legends=['Tanimoto: {:.2f}'.format(i) for i in data['toptor_tanimoto']])

In [ ]:
data

In [ ]:
data.columns

In [ ]:
# Drop the non-transformed columns
data = data.drop(columns = ['SMILES', 'mol'])

In [ ]:
sns.pairplot(data)

In [ ]:
data

In [ ]:
data = data.drop(columns = ['atom_fp', 'mac_fp','morg_fp','palen_fp','toptor_fp'])

In [ ]:
data

In [ ]:
data.columns

In [ ]:
#fuzzy c-means clustering
### Importing the required Libraries
import random
import operator
import math

In [ ]:
# Number of Attributes
num_attr = len(data.columns) - 1

# Number of Clusters to make
k = 5

# Maximum number of iterations
MAX_ITER = 100

# Number of data points
n = len(data)

# Fuzzy parameter
m = 2.00

In [ ]:
### initializing the membership matrix with random values
def initializeMembershipMatrix():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]
        summation = sum(random_num_list)
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat


In [ ]:
membership_mat = initializeMembershipMatrix()

In [ ]:
### calculating the cluster center, is done in every iteration

def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]
        denominator = sum(xraised)
        temp_num = list()
        for i in range(n):
            data_point = list(data.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, zip(*temp_num))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers

In [ ]:
#cluster_centers = calculateClusterCenter(membership_mat)
calculateClusterCenter(membership_mat)

In [ ]:
### updating the membership values using the cluster centers

def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(data.iloc[i])
        distances = [np.linalg.norm(list(map(operator.sub, x, cluster_centers[j]))) for j in range(k)]
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat

In [ ]:
def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
        cluster_labels.append(idx)
    return cluster_labels

In [ ]:
### the final fcm function, 

def fuzzyCMeansClustering():
    # Membership Matrix
    membership_mat = initializeMembershipMatrix()
    curr = 0
    while curr <= MAX_ITER:
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat = updateMembershipValue(membership_mat, cluster_centers)
        cluster_labels = getClusters(membership_mat)
        curr += 1
    
    print(membership_mat)
    return cluster_labels, cluster_centers

In [ ]:
### calling the main function and storing the final results in labels, centers

print("printing the final membership matrix")

labels, centers = fuzzyCMeansClustering()

print("printing cluster centers")
print(centers)

In [ ]:
from sklearn.metrics import silhouette_score 
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
score_fuzzy_s = silhouette_score(n,labels, metric='euclidean')
score_fuzzy_c = calinski_harabasz_score(x, labels_)
score_fuzzy_d = davies_bouldin_score(x, labels)
print('Silhouette Score: %.4f' % score_fuzzy_s)
print('Calinski Harabasz Score: %.4f' % score_fuzzy_c)
print('Davies Bouldin Score: %.4f' % score_fuzzy_d)